# WPC provisioning for MWCT2xxxS using EL2GO and MBoot

WPC provisioning serves to generate WPC Certificate Chain and insert said chain into the target.  
WPC Certificate Chain is a crucial component of WPC Qi Authentication process for wireless power transfer.

## 1. Prerequisites
1. MWCT2xxxA Board
2. NXP HSE firmware
3. NXP WPC Provisioning firmware
4. EL2GO account with access to WPC Service

If you don't have all of the prerequisites above, you may take a look into [wpc_model](./wpc_model.ipynb) notebook.  
This notebook doesn't use real HW nor EL2GO, but rather device model and EL2GO mock-up &#x1F60A;

## 2. Target board preparation
1. Flash HSE firmware using available debugger
2. Flash WPC Provisioning firmware using available debugger
3. Connect UART interface to PC
4. Verify firmwares were flashed correctly and running using `nxpdevscan --port`

In [1]:
# setup this jupyter notebook
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
%! nxpdevscan --port

nxpdevscan --port 
-------- Connected NXP UART Devices --------

Port: COM7
Type: mboot device



## 3. NXPWPC application setup
NXPWPC is the SPSDK command-line application facilitating communication between WPC Service and WPC Target.  
Out-of-the-box SPSDK provides one Service adapter using EL2GO and one Target adapter using MBoot.  
This application provides a subcommand `insert-cert` for launching the WPC Provisioning process and `get-template` for user-friendly configuration of NXPwPC app.  

In [3]:
# This command will generate a configuration file template which you then may customize to your own environment
%! nxpwpc get-template --service-type el2go --target-type mboot --family mwct2d17s --output config.yaml

nxpwpc get-template --service-type el2go --target-type mboot --family mwct2d17s --output config.yaml 
Creating config.yaml template file.


Within the `config.yaml` file please change to following:
1. `service_parameters` (configuration for WPC Service adapter)
    1. `url`: point to EL2GO instance you're registered in
    2. `qi_id`: your Qi ID registered with WPC
    3. `auth_key`: your API key obtained from EL2GO
2. `target_parameters` (configuration for WPC Target adapter)
    1. `port`: com port of your target (see output of the `nxpdevscan` command earlier)

Once the changes are done, you may run the WPC Provisioning using `nxpwpc insert-cert` command.

## 4. Running the WPC Provisioning

In [4]:
%! nxpwpc --verbose insert-cert --config config.yaml

nxpwpc --verbose insert-cert --config config.yaml 
INFO:spsdk.wpc.utils:Getting WPC ID
INFO:spsdk.wpc.target_mboot:Reading low level WPC ID
INFO:spsdk.mboot.mcuboot:Connect: identifier='uart', device=com7
INFO:spsdk.mboot.mcuboot:CMD: ReadMemory(address=0x20000000, length=136, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Successfully Received 136 from 136 Bytes
INFO:spsdk.mboot.mcuboot:Closing: identifier='uart', device=com7
INFO:spsdk.wpc.utils:Computing CSR
INFO:spsdk.wpc.target_mboot:Signing CSR-TBS data
INFO:spsdk.mboot.mcuboot:Connect: identifier='uart', device=com7
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x20000100, length=127, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(MaxPacketSize, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Max Packet Size = 32
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Successfully Send 127 out of 12

> NOTE:
If you don't want to use config file, you may specify all required settings via command line.  
Or you may use command line options to override settings in the config file.

In [5]:
%! nxpwpc insert-cert --help

nxpwpc insert-cert --help 
Usage: nxpwpc insert-cert [OPTIONS]

  Perform full WPC Cert chain flow.

    - Retrieve WPC ID from the target
    - Generate WPC cert chain on service
    - Insert WPC certificate into the target.

  Parameters for target and service may be passed using "-tp" or "-sp" options
  respectively, or via a config file ("-c"). The config file template can be
  generated using "get-template" command.

Options:
  -st, --service-type [el2go]     Name of the WPC service type.
  -sp, --service-parameters TEXT  'key=value' to set/override a service
                                  adapter setting. Can be used multiple times.
  -tt, --target-type [model|mboot]
                                  Name of the MCU target type.
  -tp, --target-parameters TEXT   'key=value' to set/override a service
                                  adapter setting. Can be used multiple times.
  -c, --config FILE               Path to the YAML/JSON configuration file.
  -s, --save-debug-data  